In [19]:
import hashlib
import json
import numpy as np
import os
import pandas as pd
import pickle
import requests

from azure.storage.queue import QueueService
from datetime import datetime, timedelta
from dotenv import load_dotenv, find_dotenv
from sklearn.ensemble import RandomForestClassifier
from os.path import join, dirname

load_dotenv(find_dotenv())

True

## Data format:

```
{
"info": 
    {
    "station": 1244, 
    "type": "TAKE", 
    "user": "charles.yu", 
    "time": 1555776040
    }, 
"type": "TAKE" // "RETURN"
}
```

In [173]:
ACCOUNT_KEY = os.getenv('ACCOUNT_KEY')
ACCOUNT_NAME = os.getenv('ACCOUNT_NAME')
MAX_MESSAGE_LIMIT = 32
QUEUE_NAME = 'batchqueue'

infile = open('station_cols','rb')
columns = pickle.load(infile)
infile = open('model','rb')
model = pickle.load(infile)

queue_service = QueueService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)

def get_queue_data():
    messages = queue_service.peek_messages(QUEUE_NAME, num_messages=32)
    deltas = { station: 0 for station in columns }
    for message in messages:
#         print(message.content)
        event = parse_result(message.content)
        if event.event_type == 'TAKE':
            deltas[event.station] -= 1
        else:
            deltas[event.station] += 1
#         queue_service.delete_message(QUEUE_NAME, message.id, message.pop_receipt)
    last_msg = parse_result(messages[-1].content)
    deltas['time'] = last_msg.time
    deltas = { k: [v] for k, v in deltas.items() }
#     print(deltas)
    df = pd.DataFrame.from_dict(deltas)
    return model.predict(df)

In [166]:
def add_dummy_task():
    task_str = '{"info": {"station": 4177, "type": "RETURN", "user": "charles.yu", "time": 1555776040}, "type": "RETURN"}'
    queue_service.put_message(QUEUE_NAME, task_str)
add_dummy_task()

In [167]:
class Event:
    def __init__(self, event_type, station, time):
        self.event_type = event_type
        self.station = station
        self.time = time

def parse_result(message):
    msg = json.loads(message)
    info = msg['info']
    return Event(info['type'], info['station'], info['time'])

In [174]:
get_queue_data()

array([4175.69])